<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/projects/teaching_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Шаблон проекта по теме "Telegram-бот для изучения иностранного языка"

Критерии оценивания

- Выполнена функция для генерации заданий на перевод (2 балла)
- Выполнена функция для генерации заданий на грамматику с множественным выбором (2 балла)
- Выполнена функция, которая проверяет правильные ответы из JSON-объекта с ответами пользователя (2 балла)
- Как минимум одна функция выполняется через Telegram-бот (2 балла)
- Функции выполняются в Telegram-боте (в *.ipynb приложены скриншоты, демонстрирующие работу бота) (2 балла)

### Шаг 1

Инициализируйте клиента HuggingFace API

Измените системный и пользовательский промпты так, чтобы получилась функция для генерации упражнений на отработку навыков перевода для любой языковой пары, например, для перевода с немецкого или французского на русский

Используйте образцы:

In [ ]:
# Инициализация клиента API
from huggingface_hub import InferenceClient

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
client = InferenceClient(model_name, token='ваш токен здесь')

In [ ]:
# Создание функции для запросов к модели
def llm_inference(user_sample):
  output = client.chat.completions.create(
          # Промпты
          messages=[
              # Основная системная инструкция
              {"role": "system", "content": "you are natural language processing guide\n"
                                            "explain the provided topic\n"
              },
              # Пользовательская инструкция
              {"role": "user",
              "content": f"explain the basics of {user_sample}"},
          ],
          stream=False,
          max_tokens=128, # Максимальная длина вывода
          temperature=0.7, # Температура
          top_p=0.1 # Объем выборки для сэмплирования
          )
  # Вывод текста, ответа модели
  return output.choices[0].get('message')['content']

### Шаг 2

Создайте новую функцию для генерации задач на проверку грамматики. Добавьте ограничение на формат ответа

Функция должна быть настроена так, чтобы модель производила JSON-объект (словарь dict) следующего вида:

```
task: "описание задачи вида "множественный выбор" по теме из пользовательского промпта"

solution: "правильный ответ"
```

Используйте сниппеты кода:

In [ ]:
# Задаем формат ответа
response_format = {
    "type": "json",
    "value": {
        "properties": {
            "task": {"type": "string"},
            "solution": {"type": "string"},
        },
        "required": ["task", "solution",]
    },
}

# Проверяем ответ модели прежде чем писать функцию
user_sample = 'Articles in English'
# Посмотрите, как мы передаем пользовательский промпт из user_sample
# с помощью f-strings форматирования строк
response = client.chat_completion(
    messages=
              # Системные и пользовательские настройки можно изменить
              {"role": "system", "content": "Generate task on the provided topic"
              },
              {"role": "user",
              "content": f"Topic: {user_sample}"},
          ],
    response_format=response_format,
    max_tokens=500,)
# Выведем результат: если результат удовлетворительный,
# начинаем писать функцию по аналогии с llm_inference из Шага 1
response.choices[0].message.content

После генерации JSON-объект нужно спарсить, т.е. превратить его в настоящий, рабочий словарь

In [ ]:
import ast
# Превращаем сгенерированный JSON (сейчас это строка, похожая на питоновский словарь)
# в настоящий питоновский словарь
result = ast.literal_eval(response.choices[0].message.content)

В этом примере мы добавили `response_format`, чтобы задать ограничение на генерацию словаря (объекта JSON)

Значение `response_format` можно адаптировать, чтобы генерировать условия задач (`task`) и их решения (`solution`)

### Шаг 3

Напишите функцию, которая принимает на вход ответ пользователя и сравнивает результат со значением ключа `solution` из JSON-объекта, который создается с помощью функции из Шага 2

Подсказка:

In [ ]:
def check_user_answer(user_answer, task_dict):
    """
    Сравнивает ответ пользователя с правильным ответом из словаря.

    :param user_answer: Ответ пользователя (строка)
    :param task_dict: Словарь с задачей и правильным ответом
    :return: True, если ответ правильный, иначе False
    """
    try:
        # Получаем правильный ответ из словаря
        correct_answer = task_dict.get('solution')

        # Проверяем, что ключ 'solution' существует
        if correct_answer is None:
            print("Ключ 'solution' не найден в словаре.")
            return False

        # Сравниваем ответ пользователя с правильным ответом
        return user_answer == correct_answer

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return False

Замените вывод `True/False` на вывод сообщений для пользователя

Добавьте вывод правильного варианта ответа, если пользователь ответил неправильно

Протестируйте функцию на сгенерированных задачах из Шага 2

### Шаг 4

Используйте туториал по работе с aiogram и домашнее задание 3, чтобы разработать функционал Telegram-бота

Для зачета проекта достаточно, чтобы чат-бот предлагал пользователю задание. Проверку решений можно реализовать *по желанию*. Это не повлияет на оценку

Добавьте 2 кнопки: "упражнения на перевод" и "задачи на грамматику"

### Шаг 4

Примените к столбцам с исходными метками и метками NLTK методы для расчета метрик Precision, Recall, F-Score, чтобы оценить качество работы NLTK

Используйте сниппеты кода:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Расчет метрик оценки
precision = precision_score(df['label'], df['predicted_label'])
recall = recall_score(df['label'], df['predicted_label'])
f1 = f1_score(df['label'], df['predicted_label'])

Визуализируйте результат с использованием матрицы ошибок

Для визуализации используйте heatmap из библиотеки seaborn

https://seaborn.pydata.org/generated/seaborn.heatmap.html

In [ ]:
from sklearn.metrics import confusion_matrix

# Расчет матрицы ошибок
conf_matrix = confusion_matrix(df['label'], df['predicted_label'])

### Шаг 5

Вставьте в ячейку ниже скриншоты, которые демонстрируют работу Telegram-бота